In [11]:
# Import Python packages
import openai
import os
import pandas as pd
import PyPDF2
import requests
import urllib

# Run the Actual Code

In [15]:
my_docs = pd.read_csv("../data/pdf_all_August2024.csv")
openai.api_key = "YOUR API KEY HERE"

In [16]:


# Generate a running list of ChatGPT response outputs
my_responses = []

# For each document...
for index, row in my_docs.iloc[758:].iterrows():
    if index == 65:
        my_responses.append("Document error...")
        continue
    elif index == 86:
        my_responses.append("Document error...")
        continue
    elif index == 759:
        my_responses.append("Document error...")
        continue
    else:
        try:
            pdf_file_obj = open("../data/samplePDFs/" + str(index) + ".pdf", "rb")
            pdf_reader = PyPDF2.PdfReader(pdf_file_obj)

            num_pages = pdf_reader.numPages
            detected_text = ""
        
        except:
            continue

    # If this is a smaller document...
    if num_pages <= 10:
        # For each page in the document...
        for page_num in range(num_pages):
            page_obj = pdf_reader.getPage(page_num)
            detected_text += page_obj.extractText() + "\n\n"
        
        # Run the ChatGPT query...
        query = "Given the text below, in places where threats are explicitly mentioned, what are the identified threats for " + row["scientificName"] + "?"
        user_msg = query + "\n\n" + detected_text

        response = openai.chat.completions.create(
            model = "gpt-4-turbo-preview",
            messages = [
                {"role": "user", "content": user_msg}
         ]
        )

        my_responses.append(response.choices[0].message.content)
    else:
         # Grab the first 10 pages
        for page_num in range(10):
            page_obj = pdf_reader.getPage(page_num)
            detected_text += page_obj.extractText() + "\n\n"

        # Run the ChatGPT query...
        query = "Given the text below, what are the identified threats for " + row["scientificName"] + "?"
        user_msg = query + "\n\n" + detected_text

        try:
            response = openai.chat.completions.create(
                model = "gpt-4-turbo-preview",
                messages = [
                    {"role": "user", "content": user_msg}
            ]
            )
            my_responses.append(response.choices[0].message.content)

        except:
            my_responses.append("Document error...")
        
    pdf_file_obj.close()
    

FileNotFoundError: [Errno 2] No such file or directory: '../data/samplePDFs/764.pdf'

In [20]:
# Write the responses to a file
my_responses_dict = {"scientificName": my_docs["scientificName"], "urls": my_docs["urls"], "Response": my_responses}
my_responses_df = pd.DataFrame(my_responses)

my_responses_df.to_csv("../data/subsample_gpt_responses4.csv")